In [1]:
import pandas as pd

In [2]:
df_all = pd.read_pickle('../input/prevfeats/data_df_proc.pkl')
df_all.shape

(18965, 649)

In [3]:
def proc(x):
    try:
        x.astype('float32')
        return True
    except ValueError:
        return False

In [4]:
cols = [col for col in df_all if proc(df_all[col]) and col != 'AdoptionSpeed']

In [6]:
df_all[cols]

,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,...,SVD_CHAR_meta_annots_top_desc_6,SVD_CHAR_meta_annots_top_desc_7,SVD_CHAR_meta_annots_top_desc_8,SVD_CHAR_meta_annots_top_desc_9,SVD_CHAR_meta_annots_top_desc_10,SVD_CHAR_meta_annots_top_desc_11,SVD_CHAR_meta_annots_top_desc_12,SVD_CHAR_meta_annots_top_desc_13,SVD_CHAR_meta_annots_top_desc_14,SVD_CHAR_meta_annots_top_desc_15
0,2,3,299,0,1,1,7,0,1,1,...,-0.134311,-0.124928,0.073656,-0.032144,0.152063,0.160691,0.044849,-0.114904,-0.068712,0.039225
1,2,1,265,0,1,1,2,0,2,2,...,-0.109716,-0.074914,0.009746,-0.050546,0.074177,0.029364,-0.034173,-0.019007,0.007265,-0.000973
2,1,1,307,0,1,2,7,0,2,2,...,-0.101082,-0.090280,-0.159905,-0.092723,-0.102873,-0.042969,0.041870,-0.069350,-0.014398,0.047513
3,1,4,307,0,2,1,2,0,2,1,...,-0.060402,0.001011,0.108416,-0.023622,-0.048951,-0.001566,-0.116973,-0.020601,0.104763,-0.075124
4,1,1,307,0,1,1,0,0,2,1,...,-0.046287,0.055240,0.005259,-0.031865,-0.013770,-0.020875,0.021946,-0.008889,-0.000683,0.046875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18960,1,5,307,307,1,1,2,0,2,1,...,-0.070227,0.131651,0.380341,0.075754,0.040518,-0.112546,0.056100,-0.111271,0.008358,-0.156620
18961,1,30,307,307,1,1,2,7,2,1,...,0.351200,0.086059,0.024756,-0.044330,-0.128756,0.110124,-0.081981,0.062493,-0.057899,-0.006701
18962,2,5,300,0,3,1,2,4,2,2,...,0.003840,0.027264,0.000049,-0.005533,-0.059212,0.014704,0.003969,0.052226,0.017335,0.017817
18963,1,24,307,0,2,5,7,0,2,2,...,0.021852,-0.003410,-0.006527,0.039624,0.034003,-0.055279,-0.007313,0.008155,-0.027556,0.035797


In [7]:
df_all = df_all[['PetID'] + cols]#df_train.columns.values.tolist()]

In [12]:
import numpy as np
import glob
import joblib
import os
from PIL import Image
import imagehash
from tqdm.auto import tqdm

paths = glob.glob('../input/petfinder-adoption-prediction/*/*jpg')

result = joblib.Parallel(n_jobs=-1, verbose=1)(joblib.delayed(lambda x: imagehash.average_hash(Image.open(x)))(path) for path in paths)


tmp = pd.DataFrame()
tmp['path'] = paths
tmp['hash'] = result
tmp['hash'] = tmp['hash'].astype(str)

tmp['is_train'] = tmp['path'].str.find('train') > -1
tmp['is_test'] = tmp['path'].str.find('test') > -1
tmp['idx'] = tmp['path'].map(lambda x: int(x.split('-')[-1][:-4]))
tmp['PetID'] = tmp['path'].map(lambda x: x.split('/')[-1].split('-')[0])

df1_train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
df1_test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')

aaa = [df1_test['PetID'].map(pd.read_csv(path, index_col='PetID')['AdoptionSpeed']).values
 for path in ['../input/pet-data/prev1st.csv']]
aaa = np.mean([t for t in aaa if not np.isnan(t.sum())], axis=0)

df1_test['AdoptionSpeed'] = aaa
df1_train = df1_train.append(df1_test)

df1 = pd.merge(tmp, df1_train, how='left', on='PetID')

df1['is_train'] = df1['is_train'].astype(np.float32)
df1['is_test'] = df1['is_train'].astype(np.float32)
df1['hash'] = df1['hash'].astype(str)
#df1 = df1[(df1['is_train'] > 0 ) | (~df1['hash'].isin(df1.loc[df1['is_train'] > 0, 'hash']))].reset_index(drop=True)
#df1.to_csv('df1_train_test_v6.csv', index=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 2000 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 4800 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 8400 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 12800 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 18000 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 24000 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 30800 tasks      | elapsed:   18.3s
[Parallel(n_jobs=-1)]: Done 38400 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 46800 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done 56000 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 66000 tasks      | elapsed:   39.2s
[Parallel(n_jobs=-1)]: Done 72776 out of 72776 | elapsed:   43.0s finished


In [13]:

df_all2 = pd.merge(df1[['PetID', 'AdoptionSpeed', 'hash']], df_all, on='PetID', how='left')

In [22]:
#df_all2.to_csv('../input/pet-data/df1_train_test_all_3.csv', index=False)


KeyboardInterrupt



In [14]:
df_all2.to_feather('../input/pet-data/df1_train_test_all_2ndplace.ftr')

In [15]:
df_all2[['AdoptionSpeed']].isna().mean()

AdoptionSpeed    0.0
dtype: float64